# Implementando Redes Neurais

In [1]:
from collections import Counter
from functools import partial
#from linear_algebra import dot
import math, random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def step_function(x):
    return 1.0 if x >= 0. else 0.

def perceptron_output(weights, bias, x):
    """returns 1 if the perceptron 'fires', 0 if not"""
    f = np.dot(weights, x) + bias
    return step_function(f)

In [3]:
x = [0,0]
weights = [2,2]
bias = -1
y = perceptron_output(weights,bias, x)
y

0.0

### Perceptron - Função OR

In [4]:
valores = np.array([[0,0],[1,0],[0,1],[1,1]])
weights = [2,2]
bias = -1
for x in valores:
    y = perceptron_output(weights,bias, x)
    #print(y)
    print(x[0], 'OR',x[1], '=', int(y))

0 OR 0 = 0
1 OR 0 = 1
0 OR 1 = 1
1 OR 1 = 1


### Perceptron - Função AND

In [5]:
valores = np.array([[0,0],[1,0],[0,1],[1,1]])
weights = [2,2]
bias = -3
for x in valores:
    y = perceptron_output(weights,bias, x)
    print(x[0], 'AND',x[1], '=', int(y))

0 AND 0 = 0
1 AND 0 = 0
0 AND 1 = 0
1 AND 1 = 1


### Perceptron - Função NOT

In [6]:
valores = np.array([[0],[1]])
weights = [-2]
bias = 1
for x in valores:
    y = perceptron_output(weights,bias, x)
    print('NOT(',x[0], ') =', int(y))

NOT( 0 ) = 1
NOT( 1 ) = 0


## Redes Neurais Feed-Forward

In [7]:
import numpy as np
import math

def sigmoid(t):
    return 1 / (1 + math.exp(-t))

def neuron_output(weights, inputs):
    return sigmoid(np.dot(weights, inputs))

def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network:

        input_with_bias = input_vector + [1]             # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

### usando feedforward

In [8]:
import numpy as np
xor_network = [ # camada oculta
                [[20, 20, -30],   # neurônio AND
                 [20, 20, -10]],  # neurônio OR
                # output layer
               [[-60, 60, -30]]
]

valores = np.array([[0,0],[1,0],[0,1],[1,1]])
weights = [2,2]
bias = -3
for dados in valores:
    x = dados[0]
    y = dados[1]
    result = feed_forward(xor_network, [x, y])
    result_int = round(result[1][0])
    print(x, 'XOR',y, '=', result_int)

0 XOR 0 = 0
1 XOR 0 = 1
0 XOR 1 = 1
1 XOR 1 = 0


## Backpropagation

In [9]:
import numpy as np
def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)

    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]

    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                      np.dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input
            
def predict(input):
    return feed_forward(network, input)[-1]

In [10]:
def patch(x, y, hatch, color):
    """return a matplotlib 'patch' object with the specified
    location, crosshatch pattern, and color"""
    return matplotlib.patches.Rectangle((x - 0.5, y - 0.5), 1, 1,
                                        hatch=hatch, fill=False, color=color)


def show_weights(neuron_idx):
    weights = network[0][neuron_idx]
    abs_weights = [abs(weight) for weight in weights]

    grid = [abs_weights[row:(row+5)] # turn the weights into a 5x5 grid
            for row in range(0,25,5)] # [weights[0:5], ..., weights[20:25]]

    ax = plt.gca() # to use hatching, we'll need the axis

    ax.imshow(grid, # here same as plt.imshow
              cmap=matplotlib.cm.binary, # use white-black color scale
              interpolation='none') # plot blocks as blocks

    # cross-hatch the negative weights
    for i in range(5): # row
        for j in range(5): # column
            if weights[5*i + j] < 0: # row i, column j = weights[5*i + j]
                # add black and white hatches, so visible whether dark or light
                ax.add_patch(patch(j, i, '/', "white"))
                ax.add_patch(patch(j, i, '\\', "black"))
    plt.show()

def make_digit(raw_digit):
    return [1 if c == '1' else 0 
    for row in raw_digit.split("\n") 
    for c in row.strip()]

## treinando a rede

In [11]:
import random

if __name__ == "__main__":

    raw_digits = [
          """11111
             1...1
             1...1
             1...1
             11111""",

          """..1..
             ..1..
             ..1..
             ..1..
             ..1..""",

          """11111
             ....1
             11111
             1....
             11111""",

          """11111
             ....1
             11111
             ....1
             11111""",

          """1...1
             1...1
             11111
             ....1
             ....1""",

          """11111
             1....
             11111
             ....1
             11111""",

          """11111
             1....
             11111
             1...1
             11111""",

          """11111
             ....1
             ....1
             ....1
             ....1""",

          """11111
             1...1
             11111
             1...1
             11111""",

          """11111
             1...1
             11111
             ....1
             11111"""]



    inputs = list(map(make_digit, raw_digits))
    
    #inputs = list(make_digit(raw_digits))

    targets = [[1 if i == j else 0 for i in range(10)]
               for j in range(10)]

    random.seed(0)   # to get repeatable results
    input_size = 25  # each input is a vector of length 25
    num_hidden = 5   # we'll have 5 neurons in the hidden layer
    output_size = 10 # we need 10 outputs for each input

    # each hidden neuron has one weight per input, plus a bias weight
    hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

    # each output neuron has one weight per hidden neuron, plus a bias weight
    output_layer = [[random.random() for __ in range(num_hidden + 1)]
                    for __ in range(output_size)]

    # the network starts out with random weights
    network = [hidden_layer, output_layer]

    # 10,000 iterations seems enough to converge
    for __ in range(10000):
        for input_vector, target_vector in zip(inputs, targets):
            backpropagate(network, input_vector, target_vector)

    for i, input in enumerate(inputs):
        outputs = predict(input)
        print(i, [round(p,2) for p in outputs])

    print(""".@@@.
...@@
..@@.
...@@
.@@@.""")
    print([round(x, 2) for x in
          predict(  [0,1,1,1,0,    # .@@@.
                     0,0,0,1,1,    # ...@@
                     0,0,1,1,0,    # ..@@.
                     0,0,0,1,1,    # ...@@
                     0,1,1,1,0])]) # .@@@.
    print()

    print(""".@@@.
@..@@
.@@@.
@..@@
.@@@.""")
    print()
    print([round(x, 2) for x in
          predict(  [0,1,1,1,0,    # .@@@.
                     1,0,0,1,1,    # @..@@
                     0,1,1,1,0,    # .@@@.
                     1,0,0,1,1,    # @..@@
                     0,1,1,1,0])]) # .@@@.
    print()

0 [0.96, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.03, 0.0]
1 [0.0, 0.96, 0.03, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2 [0.0, 0.02, 0.96, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0]
3 [0.0, 0.03, 0.0, 0.97, 0.0, 0.0, 0.0, 0.02, 0.0, 0.03]
4 [0.0, 0.01, 0.01, 0.0, 0.99, 0.0, 0.0, 0.01, 0.0, 0.0]
5 [0.0, 0.0, 0.02, 0.0, 0.0, 0.96, 0.01, 0.0, 0.01, 0.02]
6 [0.0, 0.0, 0.01, 0.0, 0.01, 0.01, 0.99, 0.0, 0.01, 0.0]
7 [0.03, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.97, 0.0, 0.0]
8 [0.03, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.96, 0.03]
9 [0.0, 0.0, 0.0, 0.01, 0.0, 0.02, 0.0, 0.0, 0.02, 0.95]
.@@@.
...@@
..@@.
...@@
.@@@.
[0.0, 0.01, 0.0, 0.97, 0.0, 0.0, 0.0, 0.02, 0.0, 0.08]

.@@@.
@..@@
.@@@.
@..@@
.@@@.

[0.0, 0.0, 0.0, 0.0, 0.0, 0.55, 0.0, 0.0, 0.92, 1.0]



---